In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import os
import glob
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 学習用データを読み取る
learning_data = pd.read_pickle('./training_DataFrame/labeled_training_dataset.pkl')
display(learning_data.head())

,rmssd,sdnn,sdsd,nn50,pnn50,mrri,mhr,ln(sdnn),stdhr,cvrr,label,label_num
filename,,,,,,,,,,,,
A00001,100.643575,65.318577,102.027740,4.0,10.526316,761.666667,79.330866,4.179276,6.890820,8.575743,N,0
A00002,290.780200,222.416471,295.066494,19.0,57.575758,893.434343,72.730171,5.404552,24.004782,24.894551,N,0
A00003,229.549405,155.065837,231.053020,32.0,41.025641,763.760684,82.196182,5.043850,19.738613,20.302935,N,0
A00004,237.096636,189.473773,241.103148,24.0,77.419355,936.021505,66.654705,5.244251,13.312296,20.242459,A,1
A00005,292.018724,211.556696,293.689064,64.0,73.563218,668.084291,98.951104,5.354493,30.106937,31.666168,A,1


In [3]:
# サンプルデータとラベルに分ける
X = learning_data.iloc[:,:10]
y = learning_data.iloc[:,-1]
display(X.head())
display(y.head())

,rmssd,sdnn,sdsd,nn50,pnn50,mrri,mhr,ln(sdnn),stdhr,cvrr
filename,,,,,,,,,,
A00001,100.643575,65.318577,102.027740,4.0,10.526316,761.666667,79.330866,4.179276,6.890820,8.575743
A00002,290.780200,222.416471,295.066494,19.0,57.575758,893.434343,72.730171,5.404552,24.004782,24.894551
A00003,229.549405,155.065837,231.053020,32.0,41.025641,763.760684,82.196182,5.043850,19.738613,20.302935
A00004,237.096636,189.473773,241.103148,24.0,77.419355,936.021505,66.654705,5.244251,13.312296,20.242459
A00005,292.018724,211.556696,293.689064,64.0,73.563218,668.084291,98.951104,5.354493,30.106937,31.666168


filename
A00001    0
A00002    0
A00003    0
A00004    1
A00005    1
Name: label_num, dtype: int64

In [4]:
# グリッドサーチによる最適パラメータ設定のために
# 学習用データを訓練データとテストデータに分ける
# 心電図の種類に偏りがないように層化サンプリングでstratify=yと指定する
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.25,
                                                   random_state=1,
                                                   stratify=y)

In [5]:
# グリッドサーチ用のパラメータを設定
# 今回の特徴量は心拍変動解析（HRV）であり、それぞれのサンプルは連続値ではないので、
# データの分割法はエントロピーを使う
parameter = {'criterion':['entropy'],
            'max_depth':[1,2,3,4,5,6,7,8,None]}

In [6]:
# グリッドサーチ
# モデルは今回は決定木を使用
GridSearch_tree = GridSearchCV(DecisionTreeClassifier(random_state=1),
                              parameter,
                              cv=5)

In [7]:
# 学習
GridSearch_tree.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'criterion': ['entropy'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, None]})

In [8]:
print(f'最良のパラメータは: {GridSearch_tree.best_estimator_}')
print(f'訓練データへのスコアは: {GridSearch_tree.score(X_train,y_train)}')
print(f'テストデータへのスコアは: {GridSearch_tree.score(X_test,y_test)}')

最良のパラメータは: DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1)
訓練データへのスコアは: 0.654043945769051
テストデータへのスコアは: 0.6442577030812325


In [9]:
# グリッドサーチの結果よりmax_depth=3として
# 再度学習用データを決定木で学習させる
# テストデータとしては予め別データフレームを使用する

# テストデータの読み込み 
test_data = pd.read_pickle('./test_DataFrame/labeled_test_dataset.pkl')
test_data = test_data.sample(frac=1)
X_test_final = test_data.iloc[:,:10]
y_test_final = test_data.iloc[:,-1]

In [10]:
tree = DecisionTreeClassifier(criterion='entropy',
                             max_depth=3,
                             random_state=1)
# 全ての学習データをシャッフルする
# X_train_final, y_train_final= train_test_split(X, y,
#                                                  test_size=None)
learning_data_final = learning_data.sample(frac=1)
X_train_final = learning_data_final.iloc[:,:10]
y_train_final = learning_data_final.iloc[:,-1]
# 全ての学習用データを使って学習
tree.fit(X_train_final, y_train_final)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1)

In [11]:
# 交差検証の平均値
score = cross_val_score(tree, X_train_final, y_train_final, cv=5)
print(np.mean(score))

0.6449387040280211


In [12]:
# 予測値
pred_test_final = tree.predict(X_test_final)

In [13]:
# 予測の精度
accuracy_score(y_test_final, pred_test_final)

0.5704697986577181

In [14]:
# 混同行列で分類状況を見る
confusion_matrix(y_test_final, pred_test_final)

array([[147,   1,   0,   0],
       [ 27,  19,   1,   0],
       [ 59,   2,   4,   0],
       [ 29,   9,   0,   0]])

In [15]:
# 適合率、再現率、F1値
print(classification_report(y_test_final, pred_test_final, digits=4,target_names=['正常心電図','心房細動','その他','雑音']))

              precision    recall  f1-score   support

       正常心電図     0.5611    0.9932    0.7171       148
        心房細動     0.6129    0.4043    0.4872        47
         その他     0.8000    0.0615    0.1143        65
          雑音     0.0000    0.0000    0.0000        38

    accuracy                         0.5705       298
   macro avg     0.4935    0.3648    0.3296       298
weighted avg     0.5498    0.5705    0.4579       298



あまり高い精度でなかったが、課題としては

・心電図の測定時間が30〜60秒と短めであり、HRVのなかでも時間領域分析データしか取得できず、周波数領域であるLF(交感神経)、HF(副交感神経）を取得するにはARIMAなど時系列分析によるデータの拡張が必要である。

・コンセントの交流雑音などの雑音波形が全く判定できていない。
デジタルフィルタ設計にさらなる工夫が必要である。

・主成分分析などで次元削減や有効な特徴量を選別するなどの工夫が必要である。

その他にも検討すべき事柄はあるが、時期を見てバージョンアップしていきたい。